In [1]:
%%writefile crop.py
import streamlit as st
import pandas as pd
import plotly.express as px
import joblib

# Page config
st.set_page_config(page_title="Crop and Livestock Dashboard", layout="wide")
st.title("\U0001F33E Crop and Livestock Data Dashboard")

# Load Data
@st.cache_data
def load_data():
    crop_df = pd.read_csv("Crop_Data.csv")
    livestock_df = pd.read_csv("Livestock_Data.csv")
    cropml_df = pd.read_csv("Crop_Data_ML.csv")
    return crop_df, livestock_df, cropml_df

crop_df, livestock_df, cropml_df = load_data()

@st.cache_resource
def load_model_and_encoder():
    model = joblib.load('random_forest_model.pkl')
    item_encoder = joblib.load('item_encoder.pkl')
    return model, item_encoder

model, item_encoder = load_model_and_encoder()

# Tabs with reordered sequence
home, tab1, tab2, tab3, tab4, tab5, tab6, tab7 = st.tabs([
    "\U0001F3E0 Overview",                      # Tab 1
    "\U0001F33E Crop Analysis",                 # Tab 2
    "\U0001F331 Crop Production Prediction",   # Tab 3 (was tab7)
    "\U0001F30D Crop Regional Comparison",     # Tab 4 (was tab3)
    "\U0001F4C8 Growth Analysis",               # Tab 5 (was tab2)
    "\U0001F404 Livestock Analysis",            # Tab 6 (was tab4)
    "\U0001F4C9 Livestock Trends",              # Tab 7 (was tab5)
    "\U0001F30E Livestock Regional Comparison"  # Tab 8 (was tab6)
])

# --- Tab 1: Home / Overview ---
with home:
    st.subheader("Crop Summary")
    year_crop_df = crop_df[crop_df["Year"] == crop_df["Year"].max()]
    col1, col2, col3 = st.columns(3)
    col1.metric("Total Crops", year_crop_df["Item"].nunique())
    col2.metric("Total Production (t)", f"{year_crop_df['Production (t)'].sum():,.0f}")
    col3.metric("Total Area Harvested (ha)", f"{year_crop_df['Area harvested (ha)'].sum():,.0f}")

    st.subheader("Livestock Summary")
    year_livestock_df = livestock_df[livestock_df["Year"] == livestock_df["Year"].max()]
    col4, col5, col6, col7 = st.columns(4)
    col4.metric("Total Stocks (No)", f"{year_livestock_df['Stocks (No)'].sum():,.0f}")
    col5.metric("Total Production (No)", f"{year_livestock_df['Production (t)'].sum():,.0f}")
    col6.metric("Total Laying (No)", f"{year_livestock_df['Laying (No)'].sum():,.0f}")
    col7.metric("Total Milk Animals (No)", f"{year_livestock_df['Milk Animals (No)'].sum():,.0f}")

# --- Tab 2: Crop Analysis ---
with tab1:
    st.subheader("Top & Bottom 10 Crops by Area Harvested")
    area_harvested = year_crop_df.groupby("Item")["Area harvested (ha)"].sum().reset_index()
    area_sorted = area_harvested.sort_values(by="Area harvested (ha)", ascending=False)
    st.plotly_chart(px.bar(area_sorted.head(10), x="Item", y="Area harvested (ha)", title="Top 10 Crops"))
    st.plotly_chart(px.bar(area_sorted.tail(10), x="Item", y="Area harvested (ha)", title="Bottom 10 Crops"))

    crop_yield = year_crop_df.groupby("Item")["Yield (kg/ha)"].mean().reset_index()
    yield_sorted = crop_yield.sort_values(by="Yield (kg/ha)", ascending=False)
    st.plotly_chart(px.bar(yield_sorted.head(10), x="Item", y="Yield (kg/ha)", title="Top 10 Crops by Yield"))
    st.plotly_chart(px.bar(yield_sorted.tail(10), x="Item", y="Yield (kg/ha)", title="Bottom 10 Crops by Yield"))

# --- Tab 3: Crop Production Prediction ---
with tab2:
    st.subheader("Predict Crop Production using ML Model")

    # Crop choices from your item_encoder
    crop_options = sorted(item_encoder.classes_)
    selected_crop = st.selectbox("Select Crop", crop_options)

    # Encode selected crop for model input
    try:
        selected_crop_encoded = item_encoder.transform([selected_crop])[0]
    except Exception:
        st.error("Selected crop is not in encoder classes.")
        selected_crop_encoded = None

    # Numeric inputs
    area_harvested = st.number_input("Area harvested (ha)", min_value=1.0, value=1000.0)
    yield_per_ha = st.number_input("Yield (kg/ha)", min_value=1.0, value=2000.0)
    year = st.slider("Year", int(cropml_df['Year'].min()), int(cropml_df['Year'].max()), value=int(cropml_df['Year'].max()))

    # For Area Code
    area_codes = sorted(cropml_df['Area Code'].unique()) if 'Area Code' in cropml_df.columns else []
    if area_codes:
        area_code = st.selectbox("Select Area Code", area_codes)
    else:
        area_code = st.number_input("Enter Area Code", min_value=0, value=0)

    if st.button("Predict Production"):
        if selected_crop_encoded is not None:
            input_df = pd.DataFrame({
                'Area harvested (ha)': [area_harvested],
                'Yield (kg/ha)': [yield_per_ha],
                'Year': [year],
                'Item_Encoded': [selected_crop_encoded],
                'Area Code': [area_code]
            })

            prediction = model.predict(input_df)[0]
            st.success(f"Predicted Production (t): {prediction:,.2f}")
        else:
            st.error("Cannot predict without proper encoding for selected crop.")

# --- Tab 4: Crop Regional Comparison ---
with tab3:
    st.subheader("Crop Production in Selected Region")
    selected_region_comparison = st.selectbox("Select Region", sorted(crop_df["Area"].unique()), key="comparison_region")
    selected_year_comparison = st.selectbox("Select Year", sorted(crop_df["Year"].unique()), key="comparison_year")

    region_crop_df = crop_df[(crop_df["Year"] == selected_year_comparison) & (crop_df["Area"] == selected_region_comparison)]
    region_production = region_crop_df.groupby("Item")["Production (t)"].sum().reset_index()
    sorted_prod = region_production.sort_values(by="Production (t)", ascending=False)
    st.plotly_chart(px.bar(sorted_prod.head(10), x="Item", y="Production (t)", title="Top 10 Crops in Region"))
    st.plotly_chart(px.bar(sorted_prod.tail(10), x="Item", y="Production (t)", title="Bottom 10 Crops in Region"))

# --- Tab 5: Growth Analysis ---
with tab4:
    st.subheader("Crop Trends Over Time")
    selected_region_growth = st.selectbox("Select Region", sorted(crop_df["Area"].unique()), key="growth_region")
    selected_crop_growth = st.selectbox("Select Crop", sorted(crop_df["Item"].unique()), key="growth_crop")

    crop_trends = crop_df[(crop_df["Area"] == selected_region_growth) & (crop_df["Item"] == selected_crop_growth)]
    if not crop_trends.empty:
        grouped = crop_trends.groupby("Year")[
            ["Production (t)", "Area harvested (ha)", "Yield (kg/ha)"]
        ].sum().reset_index()
        st.plotly_chart(px.line(grouped, x="Year", y=grouped.columns[1:], markers=True,
            title=f"Trends for {selected_crop_growth} in {selected_region_growth}"))
    else:
        st.warning("No data available for this crop in selected region.")

# --- Tab 6: Livestock Analysis ---
with tab5:
    st.subheader("Livestock Category Comparison")

    metric_option_tab4 = st.selectbox(
        "Select Metric to View",
        ["Stocks (No)", "Milk Animals (No)", "Laying (No)", "Producing Animals/Slaughtered (No)"],
        key="livestock_metric_tab4"
    )

    year_livestock_df = livestock_df[livestock_df["Year"] == livestock_df["Year"].max()]
    metric_data = year_livestock_df.groupby("Item")[metric_option_tab4].sum().reset_index()
    metric_sorted = metric_data.sort_values(by=metric_option_tab4, ascending=False)

    st.plotly_chart(px.bar(metric_sorted.head(10), x="Item", y=metric_option_tab4, title=f"Top 10 by {metric_option_tab4}"))
    st.plotly_chart(px.bar(metric_sorted.tail(10), x="Item", y=metric_option_tab4, title=f"Bottom 10 by {metric_option_tab4}"))

# --- Tab 7: Livestock Trends ---
with tab6:
    st.subheader("Livestock Trends Over Time")

    selected_region_livestock = st.selectbox("Select Region", sorted(livestock_df["Area"].unique()), key="livestock_trend_region")
    selected_livestock_item = st.selectbox("Select Livestock Item", sorted(livestock_df["Item"].unique()), key="livestock_trend_item")

    trend_df = livestock_df[
        (livestock_df["Area"] == selected_region_livestock) &
        (livestock_df["Item"] == selected_livestock_item)
    ]

    if not trend_df.empty:
        grouped_trend = trend_df.groupby("Year")[
            ["Stocks (No)", "Production (t)", "Yield (No/An)", "Laying (No)",
             "Milk Animals (No)", "Producing Animals/Slaughtered (No)"]
        ].sum().reset_index()

        st.plotly_chart(
            px.line(grouped_trend, x="Year", y=grouped_trend.columns[1:], markers=True,
                    title=f"Trends for {selected_livestock_item} in {selected_region_livestock}")
        )
    else:
        st.warning("No data available for this livestock item in selected region.")

# --- Tab 8: Livestock Regional Comparison ---
with tab7:
    st.subheader("Livestock Regional Comparison")

    col1, col2, col3 = st.columns(3)

    with col1:
        selected_area = st.selectbox("Select Region", sorted(livestock_df["Area"].unique()), key="livestock_area")

    with col2:
        selected_item = st.selectbox("Select Livestock Item", sorted(livestock_df["Item"].unique()), key="livestock_item")

    with col3:
        selected_year = st.selectbox("Select Year", sorted(livestock_df["Year"].unique()), key="livestock_year")
        metric_option_tab6 = st.selectbox(
            "Select Metric to View",
            ["Stocks (No)", "Milk Animals (No)", "Laying (No)", "Producing Animals/Slaughtered (No)"],
            key="livestock_metric_tab6"
        )

    filtered_livestock = livestock_df[
        (livestock_df["Year"] == selected_year) &
        (livestock_df["Area"] == selected_area) &
        (livestock_df["Item"] == selected_item)
    ]

    if not filtered_livestock.empty:
        grouped_data = livestock_df[
            (livestock_df["Year"] == selected_year) &
            (livestock_df["Area"] == selected_area)
        ].groupby("Item")[metric_option_tab6].sum().reset_index()

        sorted_data = grouped_data.sort_values(by=metric_option_tab6, ascending=False)

        st.plotly_chart(px.bar(sorted_data.head(10), x="Item", y=metric_option_tab6,
                               title=f"Top 10 by {metric_option_tab6} in {selected_area}"))
        st.plotly_chart(px.bar(sorted_data.tail(10), x="Item", y=metric_option_tab6,
                               title=f"Bottom 10 by {metric_option_tab6} in {selected_area}"))
    else:
        st.warning("No data available for this selection.")


Overwriting crop.py
